In [1]:
from typing import Tuple, List, Union, Any, Optional, Dict, Literal, Callable
import time
import collections
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
from torch import Tensor, tensor
import pandas as pd
import openml


from optuna_kfoldCV import evaluate_dataset_with_model, run_all_openML_with_model
from classification_param_specs import evaluate_LogisticRegression, evaluate_XGBoostClassifier

np.set_printoptions(precision=3, threshold=5) # Print options

# OpenML code

In [ ]:
# # Fetch the collection with ID 99 https://www.openml.org/search?type=study&study_type=task&id=99&sort=runs_included
# collection = openml.study.get_suite(99)
# dataset_ids = collection.data
# metadata_list = []

# # Fetch and process each dataset
# for i, dataset_id in enumerate(dataset_ids):
#     dataset = openml.datasets.get_dataset(dataset_id)
#     X, y, categorical_indicator, attribute_names = dataset.get_data(
#         target=dataset.default_target_attribute
#     )

#     #count missing values in X
#     missing_values_count = X.isnull().sum().sum()
#     print(f"Missing values in X: {missing_values_count}")

#     X = np.array(X)
#     y = np.array(y)[..., None]
#     print(X.shape)
#     print(y.shape)
    
#     # Determine if the dataset has categorical features
#     has_categorical = any(categorical_indicator)
    
#     # Extract the required metadata
#     metadata = {
#         'dataset_id': dataset.id,
#         'name': dataset.name,
#         'n_obs': int(dataset.qualities['NumberOfInstances']),
#         'n_features': int(dataset.qualities['NumberOfFeatures']),
#         '%_unique_y': len(np.unique(y))/len(y),
#         'n_unique_y': len(np.unique(y)),
#         'has_categorical': has_categorical,
#         'n_missing_values': missing_values_count,
#     }
    
#     metadata_list.append(metadata)
#     print(f" {i+1}/{len(dataset_ids)} Processed dataset {dataset.id}: {dataset.name}")

# # Create a DataFrame from the metadata list
# df_metadata = pd.DataFrame(metadata_list).sort_values('%_unique_y', ascending=False).set_index("dataset_id").sort_index()
# df_metadata.sort_values('%_unique_y', ascending=True)

# # Display the metadata DataFrame
# df_metadata

In [ ]:
# pd.set_option('display.max_rows', 72)
# df_metadata.head(72).sort_values("n_obs")

In [ ]:
# df_metadata.index

In [2]:
from optuna_kfoldCV import np_load_openml_dataset, openML_cls_ids
import numpy as np


for idx, id in enumerate(openML_cls_ids):
    X,y = np_load_openml_dataset(id, "classification")
    print("idx", idx, "id", id, "X", X.shape, "y", y.shape, np.isnan(X).sum(), np.isnan(y).sum())

# for id in df_metadata.index:
#     X,y = np_load_openml_dataset(id, "classification")
#     print("id", id, "X", X.shape, "y", y.shape, np.isnan(X).sum(), np.isnan(y).sum())

## TODO TODO TODO NEXT: investigate nans, and "Mean of empty slice". also look at less than 200 classes and check performance on reg

idx 0 id 3 X (100, 74) y (100, 1) 0 0
idx 1 id 6 X (100, 16) y (100, 26) 0 0
idx 2 id 11 X (100, 4) y (100, 3) 0 0
idx 3 id 14 X (100, 76) y (100, 10) 0 0
idx 4 id 15 X (100, 9) y (100, 1) 0 0
idx 5 id 16 X (100, 64) y (100, 10) 0 0
idx 6 id 18 X (100, 6) y (100, 10) 0 0
idx 7 id 22 X (100, 47) y (100, 10) 0 0
idx 8 id 23 X (100, 24) y (100, 3) 0 0
idx 9 id 28 X (100, 64) y (100, 10) 0 0
idx 10 id 29 X (100, 47) y (100, 1) 0 0
idx 11 id 31 X (100, 63) y (100, 1) 0 0
idx 12 id 32 X (100, 16) y (100, 10) 0 0
idx 13 id 37 X (100, 8) y (100, 1) 0 0
idx 14 id 38 X (100, 53) y (100, 1) 0 0
idx 15 id 44 X (100, 57) y (100, 1) 0 0
idx 16 id 50 X (100, 27) y (100, 1) 0 0
idx 17 id 54 X (100, 18) y (100, 4) 0 0
idx 18 id 151 X (100, 14) y (100, 1) 0 0
idx 19 id 182 X (100, 36) y (100, 6) 0 0
idx 20 id 188 X (100, 91) y (100, 5) 0 0
idx 21 id 307 X (100, 27) y (100, 11) 0 0
idx 22 id 458 X (100, 70) y (100, 4) 0 0
idx 23 id 469 X (100, 21) y (100, 6) 0 0
idx 24 id 1049 X (100, 37) y (100, 1) 0 0


# Run experiments (just for testing)

In [13]:
!python classification_param_specs.py \
    --models End2End \
    --dataset_indices 0 2 \
    --save_dir /home/nikita/Code/random-feature-boosting/save/OpenMLClassification/ \
    --n_optuna_trials 2 \
    --device cuda \
    --k_folds 5 \
    --cv_seed 42

[I 2025-01-10 12:08:31,129] A new study created in memory with name: no-name-e71f8ac1-4113-4044-8577-2980630a8d71
/home/nikita/Code/zephyrox/.conda/lib/python3.10/site-packages/optuna/distributions.py:716: UserWarning: The distribution is specified by [128, 400] and step=128, but the range is not divisible by `step`. It will be replaced by [128, 384].
  warnings.warn(
100%|███████████████████████████████████████████| 12/12 [00:00<00:00, 25.90it/s]
[I 2025-01-10 12:08:34,525] Trial 0 finished with value: 0.10872967466711998 and parameters: {'in_dim': 74, 'out_dim': 1, 'loss': 'bce', 'bottleneck_dim': 512, 'n_blocks': 2, 'hidden_dim': 431, 'lr': 0.0004570563099801456, 'end_lr_factor': 0.15751320499779725, 'n_epochs': 12, 'weight_decay': 2.9375384576328313e-06, 'batch_size': 128}. Best is trial 0 with value: 0.10872967466711998.
/home/nikita/Code/zephyrox/.conda/lib/python3.10/site-packages/optuna/distributions.py:716: UserWarning: The distribution is specified by [128, 400] and step=128,

In [7]:
!python classification_param_specs.py \
    --models LogisticRegression \
    --dataset_indices 0 2 \
    --save_dir /home/nikita/Code/random-feature-boosting/save/OpenMLClassification/ \
    --n_optuna_trials 2 \
    --device cuda \
    --k_folds 5 \
    --cv_seed 42

[I 2025-01-10 12:05:08,778] A new study created in memory with name: no-name-d0db8119-89f3-4884-9cc7-9d0c65a8e6fc
[I 2025-01-10 12:05:10,961] Trial 0 finished with value: 0.8125497698783875 and parameters: {'n_classes': 2, 'l2_lambda': 1.7670169402947936e-05}. Best is trial 0 with value: 0.8125497698783875.
[I 2025-01-10 12:05:11,037] Trial 1 finished with value: 0.4378220736980438 and parameters: {'n_classes': 2, 'l2_lambda': 0.0506157688875231}. Best is trial 1 with value: 0.4378220736980438.
[I 2025-01-10 12:05:11,050] A new study created in memory with name: no-name-1af2eb2d-f6bb-4666-a2ab-aaa4be3a73af
[I 2025-01-10 12:05:11,357] Trial 0 finished with value: 1.2218137979507446 and parameters: {'n_classes': 2, 'l2_lambda': 1.7670169402947936e-05}. Best is trial 0 with value: 1.2218137979507446.
[I 2025-01-10 12:05:11,442] Trial 1 finished with value: 0.44963467717170713 and parameters: {'n_classes': 2, 'l2_lambda': 0.0506157688875231}. Best is trial 1 with value: 0.44963467717170713

In [8]:
!python classification_param_specs.py \
    --models XGBoostClassifier \
    --dataset_indices 0 2 \
    --save_dir /home/nikita/Code/random-feature-boosting/save/OpenMLClassification/ \
    --n_optuna_trials 2 \
    --device cuda \
    --k_folds 5 \
    --cv_seed 42

[I 2025-01-10 12:05:19,085] A new study created in memory with name: no-name-1bd3b76a-9e16-420d-913e-6a4a66baa943
Error encountered during training: value 0 for Parameter num_class should be greater equal to 1
num_class: Number of output class in the multi-class classification.. Returning score 2.0 to optuna
[I 2025-01-10 12:05:19,121] Trial 0 finished with value: 2.0 and parameters: {'objective': 'multi:softmax', 'alpha': 0.0001329291894316216, 'lambda': 56.69849511478853, 'learning_rate': 0.17524101118128144, 'n_estimators': 299, 'max_depth': 2}. Best is trial 0 with value: 2.0.
Error encountered during training: value 0 for Parameter num_class should be greater equal to 1
num_class: Number of output class in the multi-class classification.. Returning score 2.0 to optuna
[I 2025-01-10 12:05:19,128] Trial 1 finished with value: 2.0 and parameters: {'objective': 'multi:softmax', 'alpha': 2.9375384576328295e-05, 'lambda': 0.0019517224641449498, 'learning_rate': 0.29621516588303487, 'n_e

In [12]:
!python classification_param_specs.py \
    --models RandomFeatureNetwork \
    --dataset_indices 0 2 \
    --save_dir /home/nikita/Code/random-feature-boosting/save/OpenMLClassification/ \
    --n_optuna_trials 2 \
    --device cuda \
    --k_folds 5 \
    --cv_seed 42

[I 2025-01-10 12:08:09,093] A new study created in memory with name: no-name-fffbd64c-bed9-4056-98df-4e827a71e8a2
[I 2025-01-10 12:08:10,512] Trial 0 finished with value: 0.5027468174695968 and parameters: {'n_layers': 0, 'in_dim': 74, 'n_classes': 2, 'upscale_type': 'SWIM', 'activation': 'tanh', 'use_batchnorm': False, 'hidden_dim': 57, 'SWIM_scale': 1.9137500362173534, 'l2_cls': 0.002465832945854913}. Best is trial 0 with value: 0.5027468174695968.
[I 2025-01-10 12:08:11,391] Trial 1 finished with value: 0.8810995578765869 and parameters: {'n_layers': 0, 'in_dim': 74, 'n_classes': 2, 'upscale_type': 'SWIM', 'activation': 'tanh', 'use_batchnorm': False, 'hidden_dim': 126, 'SWIM_scale': 0.5230326207742639, 'l2_cls': 8.62913219007186e-07}. Best is trial 0 with value: 0.5027468174695968.
[I 2025-01-10 12:08:11,533] A new study created in memory with name: no-name-843c7d62-72bd-4168-8ded-bd322446a3af
[I 2025-01-10 12:08:11,772] Trial 0 finished with value: 0.6186056733131409 and parameter

In [14]:
!python classification_param_specs.py \
    --models RFRBoost \
    --dataset_indices 0 2 \
    --save_dir /home/nikita/Code/random-feature-boosting/save/OpenMLClassification/ \
    --n_optuna_trials 2 \
    --device cuda \
    --k_folds 5 \
    --cv_seed 42

[I 2025-01-10 12:10:58,130] A new study created in memory with name: no-name-0af6995b-66ef-45d9-b69e-ae4171337321
[I 2025-01-10 12:11:00,021] Trial 0 finished with value: 0.15007127672433854 and parameters: {'in_dim': 74, 'n_classes': 2, 'feature_type': 'SWIM', 'upscale_type': 'SWIM', 'randfeat_xt_dim': 512, 'randfeat_x0_dim': 512, 'activation': 'tanh', 'use_batchnorm': True, 'do_linesearch': True, 'n_layers': 1, 'hidden_dim': 431, 'l2_cls': 0.002465832945854913, 'l2_ghat': 0.006155564318973018, 'boost_lr': 0.1432249371823025, 'SWIM_scale': 0.5229904105883546}. Best is trial 0 with value: 0.15007127672433854.
[I 2025-01-10 12:11:01,661] Trial 1 finished with value: 0.14137935936450957 and parameters: {'in_dim': 74, 'n_classes': 2, 'feature_type': 'SWIM', 'upscale_type': 'SWIM', 'randfeat_xt_dim': 512, 'randfeat_x0_dim': 512, 'activation': 'tanh', 'use_batchnorm': True, 'do_linesearch': True, 'n_layers': 1, 'hidden_dim': 321, 'l2_cls': 0.0004042872735027335, 'l2_ghat': 0.046193473743773

# join json results

In [ ]:
from pydantic.v1.utils import deep_update
import json
import os
import numpy as np
import pandas as pd

from optuna_kfoldCV import openML_reg_ids


def read_json(path):
    with open(path, "r") as f:
        return json.load(f)
    


def custom_deep_update(original, update):
    for key, value in update.items():
        if isinstance(value, dict) and key in original:
            custom_deep_update(original[key], value)
        else:
            original[key] = value



def get_joined_results_json(
        models = ["End2End", "Ridge", "XGBoostRegressor",
                  "GradientRFRBoost", "GradientRFRBoostID", 
                  "GreedyRFRBoostDense", "GreedyRFRBoostDiag", "GreedyRFRBoostScalar",
                  "RandomFeatureNetwork"],
        datasets = openML_reg_ids,
        save_dir = "/home/nikita/Code/random-feature-boosting/save/OpenMLRegression/",
        ):
    results_json = {}
    for model in models:
        for dataset in datasets:
            path = os.path.join(save_dir, f"regression_{dataset}_{model}.json")
            res = read_json(path)
            if results_json == {}:
                results_json = res
            else:
                custom_deep_update(results_json, res)
    return results_json



def join_jsons_into_array(
        results_json,
        ):
    results = []
    for dataset, dataset_results in results_json.items():
        res = []
        for model_name, model_results in dataset_results.items():
            model_res = np.stack([model_results["score_train"], model_results["score_test"], model_results["t_fit"], model_results["t_inference"]])
            res.append(model_res)
        results.append(res)
    return np.stack(results) # (n_datasets, n_models, 4, n_folds)


def results_to_df(
        models = ["End2End", "Ridge", "XGBoostRegressor", 
                  "GradientRFRBoost", "GradientRFRBoostID", 
                  "GradientRFRBoost_relu",
                  "GreedyRFRBoostDense", "GreedyRFRBoostDiag", "GreedyRFRBoostScalar",
                  "GreedyRFRBoostDense_relu", "GreedyRFRBoostDiag_relu", "GreedyRFRBoostScalar_relu",
                  "RandomFeatureNetwork", "RandomFeatureNetwork_iid",
                  "RandomFeatureNetwork_relu", "RandomFeatureNetwork_iid_relu",
                  ],
        datasets = openML_reg_ids[:],
        save_dir = "/home/nikita/Code/random-feature-boosting/save/OpenMLRegression/",
        ):
    # Load and join the JSON data
    results_json = get_joined_results_json(models, datasets, save_dir)
    results = join_jsons_into_array(results_json) # (n_datasets, n_models, 4, n_folds)
    
    # Calculate means and stds across folds
    results_mean = np.mean(results, axis=-1)  # (n_datasets, n_models, 4)
    results_std = np.std(results, axis=-1)    # (n_datasets, n_models, 4)
    
    # Create a dictionary to hold both mean and std DataFrames
    metrics = ["score_train", "score_test", "t_fit", "t_inference"]
    metric_dfs = {}
    
    # Initialize DataFrames for both mean and std metrics
    for metric in metrics:
        metric_dfs[metric] = pd.DataFrame(index=datasets, columns=models)
        metric_dfs[f"{metric}_std"] = pd.DataFrame(index=datasets, columns=models)
    
    # Populate the DataFrames for each metric
    for dataset_idx, dataset in enumerate(datasets):
        for model_idx, model in enumerate(models):
            for metric_idx, metric in enumerate(metrics):
                # Set mean value
                metric_dfs[metric].loc[dataset, model] = results_mean[dataset_idx, model_idx, metric_idx]
                # Set* std value
                metric_dfs[f"{metric}_std"].loc[dataset, model] = results_std[dataset_idx, model_idx, metric_idx]
    
    return metric_dfs


In [ ]:
for i in range(35):
    for model in ["End2End", 
                  "Ridge", "XGBoostRegressor", 
                  "GRFRBoostedXGBoostRegressor",
                  "GradientRFRBoost", #"GradientRFRBoostID", 
                  #"GradientRFRBoost_relu",
                  "GreedyRFRBoostDense", "GreedyRFRBoostDiag", "GreedyRFRBoostScalar",
                  #"GreedyRFRBoostDense_relu", "GreedyRFRBoostDiag_relu", "GreedyRFRBoostScalar_relu",
                  "RandomFeatureNetwork", 
                  "RandomFeatureNetwork_iid",
                  #"RandomFeatureNetwork_relu", "RandomFeatureNetwork_iid_relu",
                  ]:
        try:
            results = results_to_df(models=[model], datasets=[openML_reg_ids[i]])
            # print(results)
        except:
            print(f"Failed for {model} on {i}, ie {openML_reg_ids[i]}")
            pass

In [ ]:
# df = results_to_df(        
#     datasets = openML_reg_ids_noCat[[0,1,2,3,4,5,6,7,8,9,10,     13,14,15,16,17,18,19]],
#     save_dir = "/home/nikita/Code/random-feature-boosting/save/regv2_added40nlayers/OpenMLRegression/",
#     )
df = results_to_df(        
    models = ["End2End", 
              "Ridge", "XGBoostRegressor", 
              "GRFRBoostedXGBoostRegressor",
                  "GradientRFRBoost", 
                  #"GradientRFRBoostID", 
                  #"GradientRFRBoost_relu",
                  "GreedyRFRBoostDense", "GreedyRFRBoostDiag", "GreedyRFRBoostScalar",
                  #"GreedyRFRBoostDense_relu", "GreedyRFRBoostDiag_relu", "GreedyRFRBoostScalar_relu",
                  "RandomFeatureNetwork", 
                  "RandomFeatureNetwork_iid",
                  #"RandomFeatureNetwork_relu", "RandomFeatureNetwork_iid_relu",
                  ],
    datasets = openML_reg_ids[[i for i in range(35) if i not in []]], #[9, 28, 30, 33]]],
    #datasets = openML_reg_ids[:],
    save_dir = "/home/nikita/Code/random-feature-boosting/save/OpenMLRegression/",
    )

In [ ]:
df["score_test"].mean().sort_values(ascending=True)
# GradientRFRBoost_upscaleiid        0.409032
# End2End                              0.4101
# GreedyRFRBoostDense_upscaleiid     0.411854
# GreedyRFRBoostDiag_upscaleiid       0.43373
# GreedyRFRBoostScalar_upscaleiid    0.456377
# RandomFeatureNetwork_iid           0.494089
# Ridge                              0.529513
# dtype: object

In [ ]:
df["score_train"].mean().sort_values(ascending=True)

In [ ]:
df["score_test"].rank(axis=1).mean().sort_values()
# GradientRFRBoost_upscaleiid        2.400000
# GreedyRFRBoostDense_upscaleiid     2.400000
# End2End                            3.542857
# GreedyRFRBoostDiag_upscaleiid      3.685714
# GreedyRFRBoostScalar_upscaleiid    4.657143
# RandomFeatureNetwork_iid           5.371429
# Ridge                              5.942857
# dtype: float64

In [ ]:
df["score_test"].rank(axis=1).mean().sort_values()
# GradientRFRBoost_upscaleiid        2.400000
# GreedyRFRBoostDense_upscaleiid     2.400000
# End2End                            3.542857
# GreedyRFRBoostDiag_upscaleiid      3.685714
# GreedyRFRBoostScalar_upscaleiid    4.657143
# RandomFeatureNetwork_iid           5.371429
# Ridge                              5.942857
# dtype: float64




# XGBoostRegressor               2.882353
# GRFRBoostedXGBoostRegressor    3.588235
# GradientRFRBoost               4.147059
# GreedyRFRBoostDense            4.352941
# GreedyRFRBoostDiag             5.441176
# End2End                        5.794118
# RandomFeatureNetwork_iid       5.794118
# RandomFeatureNetwork           6.176471
# GreedyRFRBoostScalar           8.147059
# Ridge                          8.676471
# dtype: float64

In [ ]:
# df_old = results_to_df(        
#     models = ["End2End_cpu", 
#               "Ridge", "XGBoostRegressor", 
#                   "GradientRFRBoost", 
#                   #"GradientRFRBoostID", 
#                   #"GradientRFRBoost_relu",
#                   "GreedyRFRBoostDense", "GreedyRFRBoostDiag", "GreedyRFRBoostScalar",
#                   #"GreedyRFRBoostDense_relu", "GreedyRFRBoostDiag_relu", "GreedyRFRBoostScalar_relu",
#                   "RandomFeatureNetwork", 
#                   "RandomFeatureNetwork_iid",
#                   #"RandomFeatureNetwork_relu", "RandomFeatureNetwork_iid_relu",
#                   ],
#     #datasets = openML_reg_ids[[i for i in range(35) if i != 8]],
#     datasets = openML_reg_ids[:],
#     #save_dir = "/home/nikita/Code/random-feature-boosting/save/OpenMLRegression_swimvariable_hiddendimvariable/",
#     save_dir = "/home/nikita/Code/random-feature-boosting/save/OpenMLRegression/",
#     )

In [ ]:
from aeon.visualisation import plot_critical_difference, plot_significance
import matplotlib.pyplot as plt

# Generate the plot
plot = plot_critical_difference(df["score_test"].values,
                                df["score_test"].columns.tolist(), 
                                alpha=0.05, 
                                lower_better=True)

# Retrieve the figure and axes from the plot
fig = plot[0].figure
ax = plot[0]

# Adjust figure size
fig.set_size_inches(6, 3)

# Adjust layout
fig.tight_layout()

# Save the figures
plot[0].savefig("results/OpenMLRegression/OpenMLReg_critical_difference.eps", bbox_inches='tight')
plot[0].savefig("results/OpenMLRegression/OpenMLReg_critical_difference.png", bbox_inches='tight')

In [ ]:
import pandas as pd
import numpy as np

def create_latex_table(df):
    table = """
\\begin{table}[t]
\\caption{Test accuracies on the concentric circles task.}
\\label{tab:concentric-circles}
\\vskip 0.15in
\\begin{center}
\\begin{small}
\\begin{sc}
\\begin{tabular}{lcc}
\\toprule
Model & Mean Acc & Std Dev \\\\
\\midrule
"""
    for model_name in df.columns:
        accs = df[model_name]
        mean_acc = np.mean(accs)
        std_acc = np.std(accs)
        table += f"{model_name} & {mean_acc:.4f} & {std_acc:.4f} \\\\\n"
    
    table += """
\\bottomrule
\\end{tabular}
\\end{sc}
\\end{small}
\\end{center}
\\vskip -0.1in
\\end{table}
"""
    return table

# Example usage
# Assuming `results_df` is your pandas DataFrame
latex_table = create_latex_table(df_old["score_test"])
print(latex_table)

In [ ]:
# df2 = results_to_df(        
#     datasets = openML_reg_ids_noCat[[0,1,2  ,4,5,6,7,8,9,10,     13,14,15,16,17,18,19]],
#     save_dir = "/home/nikita/Code/random-feature-boosting/save/regv2_added40nlayers/OpenMLRegression/",
#     )

In [ ]:
# df2["score_test"].mean().sort_values(ascending=True)

In [ ]:
# df2["score_train"].mean().sort_values(ascending=True)

In [ ]:
# df2["score_test"].rank(axis=1).mean().sort_values()

In [ ]:
#number of first places
# (df2["score_test"].rank(axis=1) == 1).sum().sort_values()

In [ ]:
# XGBoostRegressor       0.389427
# End2End                0.408184
# GreedyRFBoostDense      0.43249
# GradientRFBoost        0.434696
# GradientRFBoostID      0.436364
# GreedyRFBoostDiag      0.445675
# GreedyRFBoostScalar    0.528314
# Ridge                  0.606384
# RidgeCV                0.606385
# dtype: object


# XGBoostRegressor       0.170585
# End2End                0.345058
# GreedyRFBoostDense     0.387421
# GradientRFBoost        0.400575
# GradientRFBoostID      0.405132
# GreedyRFBoostDiag       0.41455
# GreedyRFBoostScalar    0.510962
# RidgeCV                0.600331
# Ridge                   0.60034
# dtype: object


# XGBoostRegressor       2.666667
# End2End                3.333333
# GreedyRFBoostDense     3.500000
# GradientRFBoost        3.611111
# GradientRFBoostID      4.111111
# GreedyRFBoostDiag      4.777778
# GreedyRFBoostScalar    6.722222
# Ridge                  8.055556
# RidgeCV                8.222222
# dtype: float64


# Ridge                   0
# RidgeCV                 0
# GradientRFBoostID       0
# GreedyRFBoostScalar     0
# GradientRFBoost         1
# GreedyRFBoostDense      1
# GreedyRFBoostDiag       1
# End2End                 4
# XGBoostRegressor       11
# dtype: int64

# Look at small datasets

In [ ]:
less5000 = df_metadata.query("n_obs < 5000").index
less1000 = df_metadata.query("n_obs < 1000").index
less5000

In [ ]:
df["score_test"].loc[less5000].mean().sort_values(ascending=True)

In [ ]:
df["score_test"].loc[less5000].rank(axis=1).mean().sort_values(ascending=True)

In [ ]:
df["score_test"].loc[less5000].rank(axis=1)

In [ ]:
df["score_test"].loc[less1000]

# Look at distribution of params

In [ ]:
import matplotlib.pyplot as plt

def plot_param_distribution(
        models = ["End2End", "Ridge", #"XGBoostRegressor", 
                  "GradientRFRBoost", 
                  #"GradientRFRBoostID", 
                  #"GradientRFRBoost_relu",
                  "GreedyRFRBoostDense", "GreedyRFRBoostDiag", "GreedyRFRBoostScalar",
                  #"GreedyRFRBoostDense_relu", "GreedyRFRBoostDiag_relu", "GreedyRFRBoostScalar_relu",
                  "RandomFeatureNetwork", 
                  #"RandomFeatureNetwork_iid",
                  #"RandomFeatureNetwork_relu", "RandomFeatureNetwork_iid_relu",
                  ],
        datasets = openML_reg_ids[:],
        save_dir = "/home/nikita/Code/random-feature-boosting/save/OpenMLRegression/",
        # save_dir = "/home/nikita/Code/random-feature-boosting/save/regv2_added40nlayers/OpenMLRegression/",
        ):
    # Load and join the JSON data
    results_json = get_joined_results_json(models, datasets, save_dir)

    # model: list_of_param_names
    modelwise_param_names = {model: list(results_json[str(datasets[0])][model]['hyperparams'][0])
                            for model in models} 

    # model: param_name: list_of_param_values
    param_distribution = { model: {param: [] for param in param_names}
                          for model, param_names in modelwise_param_names.items()}

    #populate teh param_districution nested dict
    for dataset, dataset_results in results_json.items():
        for model_name, model_results in dataset_results.items():
            for fold in model_results["hyperparams"]:
                for param_name, param_val in fold.items():
                    param_distribution[model_name][param_name].append(param_val)

    # For each model, plot the distribution of each parameter
    for model, param_dict in param_distribution.items():
        print(f"Model: {model}")
        for param_name, param_values in param_dict.items():
            if param_name not in [
                "out_dim", "loss", "objective", "feature_type",
                "upscale_type", "sandwich_solver"
                ]:
                # Create figure with two subplots side by side
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
                
                # Linear scale plot
                ax1.hist(param_values, bins=20)
                ax1.set_title(f"{model} {param_name}\n(linear scale)")
                ax1.set_xlabel(param_name)
                ax1.set_ylabel("Count")
                
                # Log scale plot
                min_val = np.min(param_values)  # Avoid log(0)
                max_val = np.max(param_values)
                bins = np.logspace(np.log10(min_val), np.log10(max_val), 20)
                ax2.hist(param_values, bins=bins)
                ax2.set_xscale('log')
                ax2.set_title(f"{model} {param_name}\n(log scale)")
                ax2.set_xlabel(param_name)
                ax2.set_ylabel("Count")
                
                plt.tight_layout()
                plt.show()

plot_param_distribution()